Original link to the tutorial is here: http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html#model-selection-choosing-estimators-and-their-parameters

# Score and cross-validated scores

In [2]:
from sklearn import datasets,svm
digits=datasets.load_digits()
X_digits=digits.data
y_digits=digits.target
svc=svm.SVC(C=1, kernel="linear")
svc.fit(X_digits[:-100],y_digits[:-100]).score(X_digits[-100:],y_digits[-100:])

0.97999999999999998

To get a better measure of prediction accuracy (which we can use as a proxy for goodness of fit of the model), we can successively split the data in folds that we use for training and testing:

In [3]:
import numpy as np
X_folds=np.array_split(X_digits,3)
y_folds=np.array_split(y_digits,3)


In [5]:
X_folds #splits the array into set number of arrays 
# This is a numpy method

[array([[  0.,   0.,   5., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,  10.,   0.,   0.],
        [  0.,   0.,   0., ...,  16.,   9.,   0.],
        ..., 
        [  0.,   0.,   5., ...,  16.,  11.,   2.],
        [  0.,   0.,   6., ...,   0.,   0.,   0.],
        [  0.,   0.,   0., ...,  12.,   0.,   0.]]),
 array([[  0.,   0.,   1., ...,  16.,  16.,   8.],
        [  0.,   0.,  10., ...,  16.,  16.,   9.],
        [  0.,   0.,   6., ...,  16.,  15.,   3.],
        ..., 
        [  0.,   1.,  13., ...,   0.,   0.,   0.],
        [  0.,   1.,   7., ...,  12.,   2.,   0.],
        [  0.,   0.,  13., ...,   0.,   0.,   0.]]),
 array([[  0.,   0.,   0., ...,   9.,   0.,   0.],
        [  0.,   0.,   7., ...,   8.,   0.,   0.],
        [  0.,   0.,  12., ...,   0.,   0.,   0.],
        ..., 
        [  0.,   0.,   1., ...,   6.,   0.,   0.],
        [  0.,   0.,   2., ...,  12.,   0.,   0.],
        [  0.,   0.,  10., ...,  12.,   1.,   0.]])]

In [8]:
scores=list()
for k in range(3):
    # We use list to copy, in order to pop later on
    X_train=list(X_folds)
    X_test=X_train.pop(k)
    X_train=np.concatenate(X_train)
    y_train=list(y_folds)
    y_test=y_train.pop(k)
    y_train=np.concatenate(y_train)
    scores.append(svc.fit(X_train,y_train).score(X_test,y_test))
print(scores)

[0.93489148580968284, 0.95659432387312182, 0.93989983305509184]


# Cross validation generators 
Doing the same using inbuilt Scikit learn specific cross validation methods/classes. 

This example uses the split method.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
X=["a",]